<a href="https://colab.research.google.com/github/taylan-sen/CIS320/blob/main/Networking_TCP_%26_HTTP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###In this notebook, we will go over:
  1. how to use python jupyter notebooks as a python coding and sharing environment
  1. show how to make a TCP connection
  1. use TCP to interact directly with a webserver

Please complete the following activities. It will be the first steps to project 2.

### 1. jupyter notebooks
**Google colab** is a google's implementation of **jupyter**, a notebook environment for running python and saving the results, along with the ability to document it nicely.

Jupyter notebooks have the .ipynb extension. A notebook is made up of vertically stacked **cells**. Cells can be **code** cells, which contain python code, or **text** cells, which contain text written in **markdown**. **Markdown** is like a *light* version of html, it lets you insert some text formatting and links without getting bogged down with too much html tag-like gobbledygook.

A colab jupyter notebook is running in its own google-managed VM. In order to run a code cell, click on the *play* button in the top left of the cell. 

In order to launch your own google colab jupyter notebook, log into your google account in a browser and go to:

http://colab.research.google.com

The first time you use colab, you may have to *enable it*. After opening your own notebook, try making a code cell to print hello world as shown below:

In [13]:
print('hello world!')
print(5*3 +2)

hello world!
17


jupyter notebooks can also be used to send out and receive transport layer packets. In the code below, we send a message to our infamous mothership. Try implementing the following code in a new code cell:

In [4]:
import socket

s = socket.socket(type=socket.SOCK_DGRAM)
s.bind(('',12001))
s.sendto('hello from colab'.encode(), ('niagaracomputing.org', 12000))
rx, addr = s.recvfrom(2048)
print(rx.decode())
print('complete')



             ,-"""""""-.
            / o       o \ 
           /   \     /   \ 
          /     )---(     \ 
         /     ( 6 6 )     \ 
        /       | " |       \ 
       /         )=(         \ 
      /   o   .--"-"--.   o   \ 
     /    I  /  -   -  \  I    \ 
 .--(    (_}y/\       /\y{_)    )--.
(    ".___l\/__\_____/__\/l___,"    )
 \                                 /
  "-._      o O o O o O o      _,-"
      `--Y--.___________.--Y--'
         |==.___________.==| 
         `==.___________.=='

MESSAGE RECEIVED... 

complete


Colab notebooks only allow one cell to be running at a time (this was not always the case). Thus, you can't run a listener in one cell and a sender in another. 

## 2. TCP connections

Recall that TCP is a *transport layer* protocol that stands for *transmission control protocol*. Unlike UDP, which sends packets (segments) out without setting up a link, TCP is a *connection-based* protocol. In order to establish a connection, on the server we will use the **listen()** function, and on the client we will use the python **connect(_(host, port)_)** function. Recall that a *host* can be either a domain name, or an IP address. 

On your VM (not in colab) try creating tcpserver.py and tcpclient.py and sending a message back and forth:

In [ ]:
#tcpserver.py
# ECHO SERVER
import socket

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
  # SOCK_STREAM means TCP instead of UDP
  s.bind(('', 12000))
  s.listen(1)  # listen for 1 connection
  print('waiting for a connection')
  connection, client_address = s.accept()
  print('connection made from', client_address)
  while True:
    data = connection.recv(1024)
    if not data: 
      break
    print('msg received:', data.decode())
    print('sending it back')
    connection.sendall(data)


In [ ]:
#tcpclient.py
# ECHO CLIENT
import socket

HOST = 'daring.cwi.nl'    # The remote host
PORT = 50007              # The same port as used by the server
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))
    s.sendall(b'Hello, world')
    data = s.recv(1024)
print('Received', repr(data))

## 3. HTTP Protocol (HTTP/1.1)##

### request messages ###
Request messages are sent by a client to a target server. (Note: HTTP/2 and HTTP/3 have a different representation for HTTP methods and headers.)

**Request syntax**
A client sends ''request messages'' to the server, which consist of: 

1. a **request line**, consisting of the case-sensitive request method, a space , the requested URL, another space, the protocol version, a *carriage return*, and a *line feed*, e.g.:

    <code>GET /index.html HTTP/1.1</code>

1. a **HTTP request header field** (at least 1 or more headers in case of HTTP/1.1), each consisting of the case-insensitive field name, a colon, optional leading whitespace, the field value, an optional trailing whitespace and ending with a carriage return and a line feed, e.g.:

    <code>Host: www.example.com</code>

1. an **empty line**, consisting of a carriage return and a line feed;
1. an optional **HTTP message body**


In the HTTP/1.1 protocol, all header fields except <code>Host: hostname</code> are optional.


In the following code, we use TCP to connect to a webserver directly, and download the html directly. Try substituting in your VM's external IP address and downloading your own web page.

The following code has a bug or something missing in the http syntax, we will cover in next class.



In [ ]:
import socket
import time # for delay

s = socket.socket(type=socket.SOCK_STREAM)
s.connect(('niagaracomputing.org', 80))
# THERE IS SOMETHING WRONG IN THE SYNTAX OF THE NEXT LINE
s.sendall('GET / HTTP/1.1\nHOST: niagaracomputing.org\n\n'.encode())
time.sleep(1)
msg = s.recv(2048)
print(msg.decode())
print('complete')
s.close()